<a href="https://colab.research.google.com/github/kyjhub/OSS_project/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import requests
import pandas as pd
from urllib.parse import quote
from urllib.error import HTTPError, URLError
import cv2 
from google.colab.patches import cv2_imshow
import re
import numpy as np

# 영화제목 입력

In [ ]:
movie_name_ = input('어떤 영화를 검색하시겠습니까 : ')
movie_name = quote(movie_name_)

In [ ]:
url = 'https://movie.naver.com/movie/search/result.naver?query=' + movie_name + '&section=all&ie=utf8'

## 입력값에 대한 영화 목록
- 영화 제목을 완벽하게 알지 못할 때 기억나는 단어만 입력하면  
영화 제목에 입력 문자열이 있는 영화 정보와 포스터 출력

In [ ]:
movie_page = requests.get(url).content
movie_page_soup = BeautifulSoup(movie_page, 'html.parser')
#더 많은 영화보기 버튼 url 추출
more_exist = False
more_button = movie_page_soup.find('div', {'class' : 'search_all'}).find('a', {'class' : 'more_list'})
if more_button.get_text() == '더 많은 영화 보기':
  more_exist = True
  url = 'http://movie.naver.com/movie/search/result.naver?section=movie&query=' + movie_name
  #more_url = re.findall(r'\/[\w\W]+\"', str(more_button))
  #more_url = more_url[0][:-1]

In [ ]:
num=1
movie_codes = []
while True:
  movie_page = requests.get(url).content
  movie_page_soup = BeautifulSoup(movie_page, 'html.parser')
  movie_lists = movie_page_soup.find('ul', {'class' : 'search_list_1'}).find_all('li')
  for movie in movie_lists:
    try:
      movie_img_url = movie.find('img').get('src')
      urllib.request.urlretrieve(movie_img_url, './sample_data/' + str(num) + '.jpg')
      img = cv2.imread('./sample_data/' + str(num) + '.jpg')
      href = movie.find('a').get('href')
      movie_code = href.split('=')[1]
      movie_codes.append(movie_code)
      print('===================' + str(num) + '번째 영화==================')
      print(movie.dl.get_text()[1:-3].strip())
      cv2_imshow(img)
      print()
      num+=1
    except AttributeError as e:
      print('영화 표지가 없습니다.')
  try:
    url = 'http://movie.naver.com' + movie_page_soup.find('td', {'class' : 'next'}).find('a').get('href')
  except:
    break

In [ ]:
n = int(input('몇번째 영화가 찾던 영화입니까? '))
search_movie_code = movie_codes[n-1]
print(search_movie_code)

## 전체평점 다 볼 수 있는 페이지 스크래핑

- 2167 리뷰에서 HTTPERROR 413 떠서 try except 넣어줌  
- 13번째 코드 url을 for문 밖에다가 뒀더니 16번째 코드 실행할 때 url1 -> url12 -> url123이렇게 떠서 범위를 벗어나서 http error 떴던것임 13번째코드를 지금처럼 for문 안에 넣었더니 해결됨

In [ ]:
movie_review_df = pd.DataFrame(columns=("Score", "Review"))

def get_movie_review(movie_code):
  num = 0
  count = 0
  url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=' + str(movie_code) + '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1'  
    
  while True:
    try:
      print("페이지" + str(num) + " 리뷰")
      
      review_page = urllib.request.urlopen(url).read()
      review_page_soup = BeautifulSoup(review_page, 'html.parser')
      review_list = review_page_soup.select('body > div > div > div.score_result > ul > li')  #> li
      for idx, review_info in enumerate(review_list):
        score = review_info.find('em').get_text()
        review_text = review_info.find('span', {'id':'_filtered_ment_{}'.format(idx)}).get_text().strip()
        movie_review_df.loc[count] = [score, review_text]
        count += 1
      try:
        url = 'http://movie.naver.com' + review_page_soup.find('a', {'class' : 'pg_next'}).get('href')
        num += 1
      except:
        break

    except HTTPError as e:
      print(e)
      pass
  
  return movie_review_df

#### 리뷰 데이터 확인

In [ ]:
review = get_movie_review(search_movie_code)
review

## 데이터프레임 csv파일로 저장

- encoding 안 정해줬더니 파일이 깨져서 encoding 설정

In [ ]:
review.to_csv('./review.csv', encoding='utf-8-sig')

# 결측치 제거와 리뷰 데이터 개요 출력

In [ ]:
import pandas as pd
import numpy as np

#리뷰데이터 불러오기
#review = pd.read_csv('./sample_data/interstella_review.csv')
#결측치 제거 전
before = review.shape[0]

review.dropna(inplace=True)

#결측치 제거 후
after = review.shape[0]

drop_n = before - after
print('결측치 제거 행 개수 = ' + str(drop_n))

print('반올림된 리뷰 평점 평균 : ' + str(review['Score'].mean().round()))

# 불용어 제거할 때 쉽게 하기위해 리스트로 변환
review_dict = review.to_dict()
review_val = review_dict['Review'].values()
reviews = list(review_val)

print('전체 리뷰 수 : ' + str(len(reviews)))

## 한국어 형태소 분석기 설치
- MeCab 설치

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
from konlpy.tag import Mecab
tagger = Mecab()

## 영화리뷰에 관련 없는 용어 제거

In [ ]:
garbage = '영화 전 난 일 걸 뭐 줄 만 건 분 개 끝 잼 이거 번 중 듯 때 게 내 말 나 수 거 점 것 후 이 애 씨 속 뿐 밋 그 급 ㄷ 데'
garbage = garbage.split(' ')

In [ ]:
review_nouns = []

for review in reviews:
  for noun in tagger.nouns(review):
    if noun not in garbage:
      review_nouns.append(noun)

review_nouns[:10]

## 단어 빈도수

In [ ]:
from collections import Counter

review_noun_count = Counter(review_nouns)
top_review_nouns = dict(review_noun_count.most_common(100))
top_review_nouns

## 한글 폰트 설정
- 그래프 폰트 깨짐 수정하기 위해
- 출처 : https://didalsgur.tistory.com/entry/Python-Colab-%EC%82%AC%EC%9A%A9-%EC%8B%9C-%ED%95%9C%EA%B8%80-%EA%B9%A8%EC%A7%90-%ED%98%84%EC%83%81-%ED%95%B4%EA%B2%B0-feat-matplotlib

In [ ]:
def fix_font():
    # From https://HC.Dle.pw, By Jinseo Kim
    # v1.0.0
    import os
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    os.system("apt-get install -y fonts-nanum")
    os.system("fc-cache -fv")
    mpl.font_manager._rebuild()
    findfont = mpl.font_manager.fontManager.findfont
    mpl.font_manager.findfont = findfont
    mpl.backends.backend_agg.findfont = findfont
    plt.rcParams['font.family'] = "NanumBarunGothic"
    plt.rcParams['axes.unicode_minus'] = False
           
fix_font()

In [ ]:
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt

y_pos = np.arange(len(top_review_nouns))

plt.figure(figsize=(12, 24))
plt.barh(y_pos, top_review_nouns.values()) #수평 바차트
plt.title('Word Count')
plt.yticks(y_pos, top_review_nouns.keys())
plt.rc('font', family='NanumBarunGothic') 
plt.show();

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(background_color='white', font_path='./font/NanumBarunGothic.ttf')
wc.generate_from_frequencies(top_review_nouns)

In [ ]:
figure = plt.figure(figsize=(12, 12))
ax = figure.add_subplot(1, 1, 1)
ax.axis('off')
ax.imshow(wc)
plt.show()

- 다대일 
  - 입력이 여러개고 출력은 하나
  - 감정분석에 사용됨

- 언어를 숫자로 표현하는 것이 embedding

# 환경 준비
1. 라이브러리 다운로드
2. 네이버 영화평과 긍부정 데이터를 다운로드합니다 (파일보기 +  새로고침 후 확인)

* 자체 데이터셋을 사용할 경우 내용과 카테고리가 각각 content와 label 열에 들어가는 파일(아래 예시 참조)로 dataset.xlsx로 저장 후 기존 파일을 덮어쓰기 하면 됩니다. 
* 엑셀파일의 label과 content의 순서는 상관없으나 label은 0부터 시작하는 숫자로 입력하면 좋습니다. 예를들어 카테고리가 4개면 label을 0, 1, 2, 3으로 표시해주세요.

```
label  content
1      영화가 재밌다.     
1      이 영화 추천해요.     
0      지루한 영화였습니다.
...  
```

In [ ]:
!pip3 install -q transformers
!git clone https://github.com/kiyoungkim1/ReadyToUseAI

from ReadyToUseAI.src.nlp import make_sample_dataset, bert_sequence_classification
make_sample_dataset.nsmc(mode='test', text_only=False)  # mode: which datasets? 'train' or 'test'

# [Training] 
* 첨부된 샘플의 경우 약 40min 소요 (Tesla T4 GPU)
* min_sentence_length보다 긴 문장만 사용합니다.
* MAX_LEN은 모델이 인식하는 token의 길이로, 전체길이가 약 MAX_LEN의 2배보다 긴 문장은 뒷부분이 삭제됩니다 (예를들어 MAX_LEN = 128이면, 대략 길이가 256이상인 문장은 뒷부분이 무시됨).
* batch_size는 한번에 몇개의 sample을 계산하는지를 나타내며, 제한된 메모리에서 MAX_LEN을 줄이면 batch_size를 키울 수 있고, MAX_LEN를 키우면 batch_size를 줄여야 합니다. 
* epochs는 데이터셋을 몇번 반복해서 학습할지 여부이며, dataset_split은 전체 데이터 중 몇 %를 검증용 데이터셋으로 사용할지 여부입니다.

In [ ]:
CLS = bert_sequence_classification.Classification(model_name='kykim/bert-kor-base', min_sentence_length=10, MAX_LEN=128, batch_size=32, use_bert_tokenizer=True)
CLS.dataset(data_path='dataset.xlsx')
CLS.load_model(mode='train')
CLS.train(epochs=3, dataset_split=0.1)

# [Inference]
* sentences에 원하는 문장을 아래 형식과 같이 넣으면 해당하는 카테고리를 반환합니다.
* saved_model_path는 학습된 모델이 저장된 '폴더명'입니다.

In [ ]:
#sentences = ['영화 재밌어요', '영화 재미없어요', '그냥 시간떼우기용', '완전 추천작']
saved_model_path='model/saved/3'

CLS = bert_sequence_classification.Classification(model_name='kykim/bert-kor-base', min_sentence_length=10, MAX_LEN=128, batch_size=64, use_bert_tokenizer=True)
CLS.load_model(mode='inference', saved_model_path=saved_model_path)
# reviews = 크롤링한 리뷰 데이터
review_down = pd.read_csv('./review.csv', encoding='utf-8')
review_frame = review_down['Review']
before = len(review_frame)
review_frame.dropna(inplace=True)
after = len(review_frame)
review_list = review_frame.values.tolist()
print('컴퓨터에 저장된 리뷰 다운받으면서 손실된 리뷰 개수 : ', before - after)
print('총 리뷰 개수 : ', after)
#logit = CLS.inference(sentences=reviews[1000:2000])

logit = CLS.inference(sentences=review_list)
#logit는 1과 0이 나타나는 리스트
#print(logit)  # 네이버 영화평의 경우 0은 부정 카테고리, 1은 긍정 카테고리로 설정되어 있음
rate = logit.count(1)/after * 100
print(f'긍정 예측 비율 : {rate:.2f}%')